<a href="https://colab.research.google.com/github/TipsyPanda/ComplexBridges/blob/main/IPMB_Anomaly_Starter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IPMB – ML Anomaly Detection Starter Notebook
*Generated on 2025-10-13 13:12:05*

This notebook covers:
1. Data loading & basic EDA
2. Rolling-window feature engineering
3. Unsupervised anomaly detection (Isolation Forest)
4. Threshold calibration & evaluation (time-aware)
5. Minimal, stream-friendly scoring loop


## 1) Setup

In [13]:

# Install (if running elsewhere) and imports
# %pip install pandas numpy scikit-learn matplotlib scipy

import os
import json
import math
import numpy as np
import pandas as pd
from datetime import timedelta
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import precision_recall_fscore_support, average_precision_score
import matplotlib.pyplot as plt

# Matplotlib defaults (no explicit colors)
plt.rcParams["figure.figsize"] = (10, 4)

from google.colab import drive
drive.mount('/content/drive/')
# Paths
DATA_PATH = "/content/drive/MyDrive/ComplexSystemDesign/Data/"  # change if needed

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


## 2) Load with **DatetimeIndex**

In [ ]:

def load_df_with_datetimeindex(path: str, time_col: Optional[str] = "timestamp"):
    """
    Load CSV or Pickle and ensure a DatetimeIndex.
    Priority:
      1) If already DatetimeIndex -> return as is
      2) Else, if `time_col` exists -> parse and set as index
      3) Else, try to infer a datetime-like index name
    """
    if path.lower().endswith(".pkl") or path.lower().endswith(".pickle"):
        df = pd.read_pickle(path)
    else:
        df = pd.read_csv(path)
    # 1) already a DatetimeIndex
    if isinstance(df.index, pd.DatetimeIndex):
        return df.sort_index()
    # 2) has a time column to set index
    if time_col in df.columns:
        df[time_col] = pd.to_datetime(df[time_col], errors='coerce')
        df = df.dropna(subset=[time_col]).set_index(time_col).sort_index()
        return df
    # 3) try to coerce current index
    try:
        idx = pd.to_datetime(df.index, errors='raise')
        df.index = idx
        return df.sort_index()
    except Exception:
        raise ValueError("No DatetimeIndex and no time column found. Provide a 'timestamp' column or a pickle with a DatetimeIndex.")

df = load_df_with_datetimeindex(DATA_PATH, time_col="timestamp")
print(type(df.index), df.index.min(), "→", df.index.max())
print("Columns:", df.columns.tolist())
print(df.shape)
df.head()


## 3) Select a single stream for modeling (per `sensor_id`)

In [ ]:

# Pick the first available sensor_id within the first sensor_type (customize as needed)
first_type = df['sensor_type'].dropna().unique()[0]
first_sensor = df.loc[df['sensor_type']==first_type, 'sensor_id'].dropna().unique()[0]

sub = df[(df['sensor_type']==first_type) & (df['sensor_id']==first_sensor)].copy()
sub = sub.sort_index()
print("Working stream:", first_type, first_sensor, "rows:", len(sub))

# Quick raw plot
fig = plt.figure()
plt.plot(sub.index, sub['value'])
plt.title(f"Raw values – {first_sensor} ({first_type})")
plt.xlabel("time"); plt.ylabel("value")
plt.show()

# Optional: overlay rule anomalies as vertical lines (if present)
if 'anomaly' in sub.columns:
    fig = plt.figure()
    plt.plot(sub.index, sub['value'])
    t_anom = sub.index[sub['anomaly']==1]
    for t in t_anom:
        plt.axvline(t, linestyle='--', linewidth=0.8)
    plt.title(f"Rule anomalies overlay – {first_sensor}")
    plt.xlabel("time"); plt.ylabel("value")
    plt.show()


## 4) Rolling-window feature engineering (uses index)

In [ ]:

def infer_base_freq(idx: pd.DatetimeIndex) -> str:
    """Infer a base frequency string from a DatetimeIndex."""
    # Try pandas inference first
    guessed = pd.infer_freq(idx)
    if guessed is not None:
        return guessed
    # Fallback to median delta in milliseconds
    deltas = idx.to_series().diff().dropna().dt.total_seconds()
    if len(deltas) == 0:
        return "1S"
    ms = int(np.median(deltas) * 1000)
    ms = max(ms, 1)  # at least 1ms
    return f"{ms}L"  # 'L' = milliseconds

def make_rolling_features_index(ts: pd.DataFrame, window: str = '30s', step: str = '15s'):
    """
    Build rolling-window features assuming ts.index is a DatetimeIndex.
    Keeps time as the index; returns a DataFrame with 't_center' as a datetime column.
    """
    if not isinstance(ts.index, pd.DatetimeIndex):
        raise TypeError("Expected a DatetimeIndex")
    ts = ts.sort_index()

    base_freq = infer_base_freq(ts.index)
    # Resample to fixed grid, forward-fill short gaps
    ts = ts.resample(base_freq).ffill()

    start, end = ts.index.min(), ts.index.max()
    out = []
    cur = start
    w = pd.to_timedelta(window)
    s = pd.to_timedelta(step)

    while cur + w <= end:
        win = ts.loc[cur:cur+w]
        if len(win) < 3:
            cur += s
            continue

        v = win['value'].values
        mean = float(np.mean(v))
        std = float(np.std(v, ddof=1)) if len(v) > 1 else 0.0
        rms = float(np.sqrt(np.mean(v**2)))
        p2p = float(np.max(v) - np.min(v))
        skew = float(pd.Series(v).skew()) if len(v) > 2 else 0.0
        kurt = float(pd.Series(v).kurtosis()) if len(v) > 3 else 0.0
        x = np.arange(len(v))
        slope = float(np.polyfit(x, v, 1)[0]) if len(v) > 1 else 0.0
        dv = np.diff(v)
        adiff_mean = float(np.mean(np.abs(dv))) if len(dv) else 0.0

        ctx = {}
        if 'traffic_load_proxy' in win.columns:
            tl = win['traffic_load_proxy'].values
            ctx['ctx_tl_mean'] = float(np.mean(tl))
            ctx['ctx_tl_std']  = float(np.std(tl, ddof=1)) if len(tl)>1 else 0.0
        if 'rule_threshold' in win.columns:
            ctx['ctx_rule_thr'] = float(np.median(win['rule_threshold']))

        label = None
        if 'anomaly' in win.columns:
            label = int((win['anomaly'] == 1).any())

        t_center = cur + w/2
        row = {
            't_center': t_center,
            'f_mean': mean, 'f_std': std, 'f_rms': rms, 'f_p2p': p2p,
            'f_skew': skew, 'f_kurt': kurt, 'f_slope': slope, 'f_adiff_mean': adiff_mean,
            **ctx
        }
        if label is not None:
            row['label_rule'] = label
        out.append(row)

        cur += s

    feats = pd.DataFrame(out).sort_values('t_center').reset_index(drop=True)
    return feats

feats = make_rolling_features_index(
    sub[['value','traffic_load_proxy','rule_threshold','anomaly']].copy(),
    window='30s', step='15s'
)
print(feats.shape)
feats.head()


## 5) Temporal split and scaling

In [ ]:

cut = int(len(feats) * 0.7)
train = feats.iloc[:cut].copy()
test  = feats.iloc[cut:].copy()

feature_cols = [c for c in feats.columns if c.startswith('f_') or c.startswith('ctx_')]
label_col = 'label_rule' if 'label_rule' in feats.columns else None

from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
Xs_tr = scaler.fit_transform(train[feature_cols])
Xs_te = scaler.transform(test[feature_cols])

import joblib
joblib.dump(scaler, os.path.join(ARTIFACT_DIR, "scaler.joblib"))
print("Saved:", os.path.join(ARTIFACT_DIR, "scaler.joblib"))


## 6) Isolation Forest (unsupervised)

In [ ]:

from sklearn.ensemble import IsolationForest

iso = IsolationForest(
    n_estimators=300,
    contamination='auto',
    random_state=42,
    n_jobs=-1
)
iso.fit(Xs_tr)

score_tr = iso.decision_function(Xs_tr)
score_te = iso.decision_function(Xs_te)

import joblib
joblib.dump(iso, os.path.join(ARTIFACT_DIR, "isoforest.pkl"))
print("Saved:", os.path.join(ARTIFACT_DIR, "isoforest.pkl"))


## 7) Threshold calibration

In [ ]:

import numpy as np, json

def pick_threshold(scores, quantile=0.005):
    return float(np.quantile(scores, quantile))

thr = pick_threshold(score_tr, quantile=0.005)
print("Chosen threshold:", thr)

with open(os.path.join(ARTIFACT_DIR, "threshold.json"), "w") as f:
    json.dump({"score_threshold": thr}, f)
print("Saved:", os.path.join(ARTIFACT_DIR, "threshold.json"))


## 8) Evaluation & visuals

In [ ]:

from sklearn.metrics import precision_recall_fscore_support, average_precision_score
import matplotlib.pyplot as plt

alerts_te = (score_te < thr).astype(int)

if label_col is not None and label_col in test.columns:
    y_true = test[label_col].values
    anomaly_score_te = -score_te
    ap = average_precision_score(y_true, anomaly_score_te)
    p, r, f1, _ = precision_recall_fscore_support(y_true, alerts_te, average='binary', zero_division=0)
    print(f"AP: {ap:.3f}  Precision: {p:.3f}  Recall: {r:.3f}  F1: {f1:.3f}")
else:
    print("No labels available; showing score plots only.")

# Score timeline
fig = plt.figure()
plt.plot(test['t_center'], score_te)
plt.axhline(thr, linestyle='--')
plt.title("IsolationForest score (higher=more normal)")
plt.xlabel("time"); plt.ylabel("score")
plt.show()

# Alerts (binary)
fig = plt.figure()
plt.plot(test['t_center'], alerts_te, drawstyle='steps-post')
plt.title("Alerts (test)")
plt.xlabel("time"); plt.ylabel("alert")
plt.show()


## 9) Mini scoring loop (DatetimeIndex pipeline)

In [ ]:

events = []
for t, s, a in zip(test['t_center'], score_te, (score_te < thr).astype(int)):
    if a == 1:
        events.append({"t": str(t), "event": "ALERT", "score": float(s)})
len(events), events[:5]


## 10) Notes & next steps
- Resampling is driven by the DatetimeIndex (no `timestamp` column).
- Extend features with FFT band powers for vibration streams.
- Consider per-sensor models for tighter boundaries.
- Add drift monitoring with rolling median/MAD of scores.
- Wire artifacts into a Streamlit dashboard.
